In [1]:
import pandas as pd
import requests
import json
import glob
import os

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMC0wNy0xMVQwNzo1Mjo0Mi40NjBaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE1OTQ0NTM5NjIsImV4cCI6MTYwMTc5Nzk2MiwiYXRfaGFzaCI6InRESkRxZUJnRWpBa2N5dFNKUjdCZkEiLCJub25jZSI6IkFmTy5YSn5ZfklOaHh5M2dUZ0hyTC50QUdoRmt6ZWJjIn0.P01TNTizPXHobT3Yt8Md3gTUNkK7I9opZMilZ_iAHsU6-YHwuRQo0jjIdyqtsTvxhBzGpCsEeEez7h2C1kOnrsIF57m3O6DRnRoEeNyB52vP--z7A-zx1KrY2EuawqAVeWc5-mQCcYohSWmx75yhi0bQD3PoPbP6jNxUnmYT3O11_AbnZAM5UkB6jaAcYpsfUVXl6cB_z_R1iuWJvt5fSHJg53LPtL4waV61YEaVCbl8h9ON1Cz13-vsd_2dndxwC_CqA9gdLEw2HUF195ebtDMvVN6raYojvRZGg1EI9HVcCK0sP4q6VUspGDMZm2L97UaKCw7SYwA5zQ6WJ46bJw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"989eaff39d246ad95cf758e038a4c8af22ed019f","refresh_token":"4e1956a42605cb4862f6a090ec144a54cb03f647"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,151,293957,Brodie,Grundy,COL,RUC
0,469,290528,Max,Gawn,MEL,RUC
0,755,295467,Jack,Macrae,WBD,MID
0,76,293535,Lachie,Neale,BRL,MID
0,244,291570,Nat,Fyfe,FRE,MID
...,...,...,...,...,...,...
0,818,1016214,Ayce,Taylor,ADE,DEF
0,821,1016097,Henry,Crauford,ESS,RUC
0,822,1020224,Tom,Hird,ESS,MID
0,825,297668,Nicholas,Reid,WCE,FWD


## LADDER AND FIXTURE DATA

In [5]:
# specify round
round_number = 10

s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(round_number), headers=headers)
match_json = json.loads(s.text)

## COACH IDs AND NAMES

In [6]:
df_coaches = pd.DataFrame()

for coach in match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])

df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,180650,33301,Jordan,"What's Up, Doch?"
0,86078,9806,Lester,HoweIMetYourMother
0,45168,5524,Luke,The Dragon's Laird
0,300791,182,Paul,Coronaviney
0,8258,9856,Simon,It's always Sonny
0,10857,189,Anthony,The Full Bonty
0,21562,186,Mark,GetRichOrDieTryin
0,71052,9867,James,Bad Boys For Fyfe


## PLAYER MATCH RESULTS

In [17]:
def get_player_score_data(on_field_status):
    df_player_results = pd.DataFrame()
    if on_field_status == True:
        scoring_tag = 'scoring'
    else:
        scoring_tag = 'nonscoring'
    df_player_line = pd.DataFrame()
    for player in coach['userTeam']['scores'][scoring_tag]:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results
    

df_player_match_results = pd.DataFrame()

for coach in match_json['ladder']:
    df_player_line = pd.DataFrame()
    df_on_field_players = get_player_score_data(True)
    df_off_field_players = get_player_score_data(False)
    df_all_players = pd.concat([df_on_field_players, df_off_field_players])
    df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number'] = round_number
df_player_match_results['round_number_alt'] = "R" + str(round_number)
df_player_match_results

,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number,round_number_alt
0,71052,9867,James,Bad Boys For Fyfe,587,MID,134,True,270919,Tom,Rockliff,PTA,MID,10,R10
1,71052,9867,James,Bad Boys For Fyfe,645,FWD,116,True,992374,Dan,Butler,STK,FWD,10,R10
2,71052,9867,James,Bad Boys For Fyfe,755,MID,113,True,295467,Jack,Macrae,WBD,MID,10,R10
3,71052,9867,James,Bad Boys For Fyfe,597,FWD,112,True,1000223,Liam,Baker,RIC,DEF FWD,10,R10
4,71052,9867,James,Bad Boys For Fyfe,469,RUC,107,True,290528,Max,Gawn,MEL,RUC,10,R10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,71052,9867,James,Bad Boys For Fyfe,219,MID,55,True,291783,Dylan,Shiel,ESS,MID,10,R10
172,71052,9867,James,Bad Boys For Fyfe,121,DEF,0,False,994539,Sam,Petrevski-Seton,CAR,DEF MID,10,R10
173,71052,9867,James,Bad Boys For Fyfe,244,MID,0,False,291570,Nat,Fyfe,FRE,MID,10,R10
174,71052,9867,James,Bad Boys For Fyfe,816,MID,0,False,292128,Elliot,Yeo,WCE,MID,10,R10


In [8]:
df_player_match_results.to_csv('outputs/sc_rounds/round_{}_player_match_results.csv'.format(round_number))

In [9]:
def get_results_dataframe(path):
    data = []
    data_list = []
    df_combined_player_match_results = pd.DataFrame()
    for item in glob.glob(path):
        file = open(item, 'r')
        name = file.name
        data = pd.read_csv(name, index_col=0)
        df_combined_player_match_results = pd.concat([df_combined_player_match_results, data])
    return df_combined_player_match_results


path = "outputs/sc_rounds/*player_match_results.csv"

df_combined_player_match_results = get_results_dataframe(path)
df_combined_player_match_results.to_csv('outputs/sc_rounds/player_match_results_COMBINED.csv')
df_combined_player_match_results

,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number,round_number_alt
0,180650,33301,Jordan,"What's Up, Doch?",587,MID,134,True,270919,Tom,Rockliff,PTA,MID,10,R10
1,180650,33301,Jordan,"What's Up, Doch?",645,FWD,116,True,992374,Dan,Butler,STK,FWD,10,R10
2,180650,33301,Jordan,"What's Up, Doch?",755,MID,113,True,295467,Jack,Macrae,WBD,MID,10,R10
3,180650,33301,Jordan,"What's Up, Doch?",597,FWD,112,True,1000223,Liam,Baker,RIC,DEF FWD,10,R10
4,180650,33301,Jordan,"What's Up, Doch?",469,RUC,107,True,290528,Max,Gawn,MEL,RUC,10,R10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,10857,189,Anthony,The Full Bonty,576,NaN,45,True,997142,Peter,Ladhams,PTA,NaN,9,R9
172,10857,189,Anthony,The Full Bonty,322,NaN,73,False,298524,Lachie,Weller,GCS,NaN,9,R9
173,10857,189,Anthony,The Full Bonty,286,NaN,87,False,293713,Brandon,Ellis,GCS,NaN,9,R9
174,10857,189,Anthony,The Full Bonty,413,NaN,0,False,280109,Callan,Ward,GWS,NaN,9,R9


## LADDER

In [20]:
df_player_match_results = pd.DataFrame()

df_ladder = pd.DataFrame()

for coach in match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
            'wins': coach['wins'],
            'draws': coach['draws'],
            'losses': coach['losses'],
            'points_for': coach['points_for'],
            'points_against': coach['points_against'],
            'league_points': coach['points'],
            'position': coach['position']
        }
    )
    df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,180650,33301,Jordan,"What's Up, Doch?",7,0,3,16056,15579,28,1
0,86078,9806,Lester,HoweIMetYourMother,6,0,4,16686,16199,24,2
0,45168,5524,Luke,The Dragon's Laird,6,0,4,16563,16483,24,3
0,300791,182,Paul,Coronaviney,5,0,5,15839,16167,20,4
0,8258,9856,Simon,It's always Sonny,4,0,6,16259,15947,16,5
0,10857,189,Anthony,The Full Bonty,4,0,6,16282,16115,16,6
0,21562,186,Mark,GetRichOrDieTryin,4,0,6,16257,16397,16,7
0,71052,9867,James,Bad Boys For Fyfe,4,0,6,15309,16364,16,8


## FIXTURE RESULTS

In [10]:
df_fixture_results = pd.DataFrame()
for result in match_json['fixtures']:
    df_line = pd.DataFrame(
        {
            'round_number': [result['round']],
            'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
            'home_team_id': [result['user_team1']['id']],
            'home_team_name': [result['user_team1']['teamname']],
            'home_team_coach': [result['user_team1']['user']['first_name']],
            'home_team_points': [result['user_team1']['stats'][0]['points']],
            'away_team_id': [result['user_team2']['id']],
            'away_team_name': [result['user_team2']['teamname']],
            'away_team_coach': [result['user_team2']['user']['first_name']],
            'away_team_points': [result['user_team2']['stats'][0]['points']],
        }
    )
    df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,10,Paul vs Lester,182,Coronaviney,Paul,1583,9806,HoweIMetYourMother,Lester,1548
0,10,James vs Anthony,9867,Bad Boys For Fyfe,James,1516,189,The Full Bonty,Anthony,1792
0,10,Jordan vs Mark,33301,"What's Up, Doch?",Jordan,1598,186,GetRichOrDieTryin,Mark,1437
0,10,Simon vs Luke,9856,It's always Sonny,Simon,1694,5524,The Dragon's Laird,Luke,1318


In [11]:
df_fixture_results.to_csv('outputs/sc_rounds/round_{}_fixture_results.csv'.format(round_number))

In [12]:
path = "outputs/sc_rounds/*fixture_results.csv"
df_combined_fixture_results = get_results_dataframe(path)
df_combined_fixture_results.to_csv('outputs/sc_rounds/fixture_results_COMBINED.csv')
df_combined_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,10,Paul vs Lester,182,Coronaviney,Paul,1583,9806,HoweIMetYourMother,Lester,1548
0,10,James vs Anthony,9867,Bad Boys For Fyfe,James,1516,189,The Full Bonty,Anthony,1792
0,10,Jordan vs Mark,33301,"What's Up, Doch?",Jordan,1598,186,GetRichOrDieTryin,Mark,1437
0,10,Simon vs Luke,9856,It's always Sonny,Simon,1694,5524,The Dragon's Laird,Luke,1318
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476
0,1,James vs Jordan,9867,Bad Boys For Fyfe,James,1623,33301,"What's Up, Doch?",Jordan,1721
0,2,Paul vs Anthony,182,Coronaviney,Paul,1676,189,The Full Bonty,Anthony,1653
0,2,Lester vs Mark,9806,HoweIMetYourMother,Lester,1658,186,GetRichOrDieTryin,Mark,1642


### Rearrange fixture data to display by team + opposition scores

In [13]:
df_home_fixture_results = df_combined_fixture_results
df_away_fixture_results = df_combined_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team.to_csv('outputs/sc_rounds/fixture_results_COMBINED_BY_TEAM.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points
0,10,Paul vs Lester,182,Coronaviney,Paul,1583,9806,HoweIMetYourMother,Lester,1548
0,10,James vs Anthony,9867,Bad Boys For Fyfe,James,1516,189,The Full Bonty,Anthony,1792
0,10,Jordan vs Mark,33301,"What's Up, Doch?",Jordan,1598,186,GetRichOrDieTryin,Mark,1437
0,10,Simon vs Luke,9856,It's always Sonny,Simon,1694,5524,The Dragon's Laird,Luke,1318
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657
...,...,...,...,...,...,...,...,...,...,...
0,8,James vs Jordan,33301,"What's Up, Doch?",Jordan,1630,9867,Bad Boys For Fyfe,James,1639
0,9,Paul vs Anthony,189,The Full Bonty,Anthony,1630,182,Coronaviney,Paul,1637
0,9,Lester vs Mark,186,GetRichOrDieTryin,Mark,1667,9806,HoweIMetYourMother,Lester,1875
0,9,James vs Luke,5524,The Dragon's Laird,Luke,1743,9867,Bad Boys For Fyfe,James,1497


## CURRENTLY LISTED TEAMS

In [14]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,151,owner,186.0,NaN,293957,Brodie,Grundy,COL,RUC,21562,186,Mark,GetRichOrDieTryin
1,469,owner,33301.0,NaN,290528,Max,Gawn,MEL,RUC,180650,33301,Jordan,"What's Up, Doch?"
2,755,owner,33301.0,NaN,295467,Jack,Macrae,WBD,MID,180650,33301,Jordan,"What's Up, Doch?"
3,76,owner,5524.0,NaN,293535,Lachie,Neale,BRL,MID,45168,5524,Luke,The Dragon's Laird
4,244,owner,9867.0,NaN,291570,Nat,Fyfe,FRE,MID,71052,9867,James,Bad Boys For Fyfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,598,owner,189.0,NaN,1002245,Noah,Balta,RIC,RUC FWD,10857,189,Anthony,The Full Bonty
172,174,owner,182.0,NaN,1003029,Brayden,Sier,COL,MID,300791,182,Paul,Coronaviney
173,645,owner,33301.0,NaN,992374,Dan,Butler,STK,FWD,180650,33301,Jordan,"What's Up, Doch?"
174,263,owner,9806.0,NaN,1009420,Caleb,Serong,FRE,MID,86078,9806,Lester,HoweIMetYourMother


## CURRENT FREE AGENTS + WAIVER

In [15]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position
32,513,free_agent,NaN,NaN,280921,Ben,Cunnington,NTH,MID
43,5,free_agent,NaN,NaN,294307,Brad,Crouch,ADE,MID
45,676,free_agent,NaN,NaN,293846,Sebastian,Ross,STK,MID
47,108,free_agent,NaN,NaN,261299,Matthew,Kreuzer,CAR,RUC
53,116,free_agent,NaN,NaN,297907,Nic,Newman,CAR,DEF
...,...,...,...,...,...,...,...,...,...
821,818,free_agent,NaN,NaN,1016214,Ayce,Taylor,ADE,DEF
822,821,free_agent,NaN,NaN,1016097,Henry,Crauford,ESS,RUC
823,822,free_agent,NaN,NaN,1020224,Tom,Hird,ESS,MID
824,825,free_agent,NaN,NaN,297668,Nicholas,Reid,WCE,FWD


In [16]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')